# Assingment 2 Advanced Machine Learning 

Overview of the Assignment
You need to work with three image datasets that contain class-conditional random label noise:

FashionMNIST0.3.npz - Known transition matrix
FashionMNIST0.6.npz - Known transition matrix
CIFAR.npz - Unknown transition matrix (requires estimation)

Your main tasks are:

Implement at least two classification algorithms robust to label noise
Build a transition matrix estimator for the third dataset
Compare the robustness of your algorithms

In [2]:
#%pip install tensorflow

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install scikit-learn

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 26.9 MB/s eta 0:00:00a 0:00:01
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 30.8 MB/s eta 0:00:00a 0:00:01
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [43]:
%pip install seaborn
%pip install numpy
%pip install scikit-learn
%pip install matplotlib

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 15.4 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [37]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

from tensorflow.keras import layers, models
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split


In [4]:
# Load dataset
dataset = np.load('/Users/Aidar Alimbayev/Documents/projects_aidar_mac/ml805_labs/Assingment 2 /datasets/CIFAR10.npz')

In [5]:
dataset

NpzFile '/Users/Aidar Alimbayev/Documents/projects_aidar_mac/ml805_labs/Assingment 2 /datasets/CIFAR10.npz' with keys: X_tr, S_tr, X_ts, Y_ts

In [6]:
# Load dataset
X_train = dataset['X_tr']  # Training features
S_train = dataset['S_tr']  # Noisy labels for training
X_test = dataset['X_ts']  # Test features
Y_test = dataset['Y_ts']  # Clean labels for testing

In [7]:
X_train, X_val, S_train, S_val = train_test_split(X_train, S_train, test_size=0.2, random_state=42)


In [8]:
# Data preprocessing (normalize, reshape if needed)
# For images, you might want to normalize the pixel values to [0,1]
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# # Reshape if needed
# X_train = X_train.reshape(-1, 32, 32, 3)
# X_val = X_val.reshape(-1, 32, 32, 3)
# X_ts = X_ts.reshape(-1, 32, 32, 3)


In [53]:
dataset_f = np.load('/Users/Aidar Alimbayev/Documents/projects_aidar_mac/ml805_labs/Assingment 2 /datasets/FashionMNIST0.3.npz')
X_train = dataset_f['X_tr']  # Training features
S_train = dataset_f['S_tr']  # Noisy labels for training
X_test = dataset_f['X_ts']  # Test features
Y_test = dataset_f['Y_ts']  # Clean labels for testing




In [16]:
# #X_train
# X_val

In [9]:
def estimate_transition_matrix(features, noisy_labels, num_classes=4):
    # Train a base classifier (e.g., neural network, random forest)
    model = train_base_classifier(features, noisy_labels)
    
    # Get predicted probabilities on training data
    pred_probs = model.predict_proba(features)
    
    # Estimate transition matrix using predicted probabilities
    T_estimated = np.zeros((num_classes, num_classes))
    
    # Implementation details here...
    
    return T_estimated

# Some example of CIFAR CNN

In [38]:
def create_cifar_cnn():
    """
    Creates a deeper CNN suitable for CIFAR classification.
    """
    model = models.Sequential([
        # Input layer
        layers.Input(shape=(32, 32, 3)),
        
        # First convolutional block
        layers.Conv2D(64, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(64, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.2),
        
        # Second convolutional block
        layers.Conv2D(128, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(128, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.3),
        
        # Third convolutional block
        layers.Conv2D(256, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(256, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.4),
        
        # Fully connected layers
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(4)  # 4 classes
    ])
    
    return model

In [39]:
def forward_correction_loss(y_true, y_pred, T):
    """
    Forward correction loss using transition matrix T
    """
    # Convert one-hot to sparse if needed
    if len(y_true.shape) > 1:
        y_true = tf.argmax(y_true, axis=1)
    
    # Get predicted probabilities
    pred_probs = tf.nn.softmax(y_pred)
    
    # Apply transition matrix correction
    corrected_probs = tf.matmul(pred_probs, tf.constant(T, dtype=tf.float32))
    
    # Calculate cross-entropy loss with corrected probabilities
    loss = tf.nn.sparse_categorical_crossentropy(y_true, corrected_probs)
    
    return tf.reduce_mean(loss)

# Create and compile model with custom loss
model = create_cifar_cnn()
model.compile(
    optimizer='adam',
    loss=lambda y_true, y_pred: forward_correction_loss(y_true, y_pred, T),
    metrics=['accuracy']
)

In [40]:
import numpy as np
from sklearn.model_selection import train_test_split

def evaluate_classifier(classifier, Xtr, Str, Xts, Yts, n_runs=3):
    accuracies = []
    
    for i in range(n_runs):
        # Random seed for reproducibility but different splits
        seed = 42 + i
        
        # Split data into train/validation
        X_train, X_val, S_train, S_val = train_test_split(
            Xtr, Str, test_size=0.2, random_state=seed
        )
        
        # Train the classifier
        classifier.fit(X_train, S_train, X_val, S_val)
        
        # Evaluate on test set
        accuracy = classifier.evaluate(Xts, Yts)
        accuracies.append(accuracy)
    
    # Calculate mean and standard deviation
    mean_acc = np.mean(accuracies)
    std_acc = np.std(accuracies)
    
    return mean_acc, std_acc

In [44]:


class NoiseRobustClassifier:
    def __init__(self, name, model_builder, transition_matrix=None, use_correction=True):
        """
        A wrapper for noise-robust classifiers.
        
        Args:
            name: Name of the classifier for reporting
            model_builder: Function that returns a TF/Keras model
            transition_matrix: The noise transition matrix if known
            use_correction: Whether to use transition matrix correction
        """
        self.name = name
        self.model_builder = model_builder
        self.transition_matrix = transition_matrix
        self.use_correction = use_correction
        self.model = None
        self.history = None
    
    def forward_correction_loss(self, y_true, y_pred):
        """Forward correction loss using transition matrix"""
        # Handle case when y_true is one-hot encoded
        if len(tf.shape(y_true)) > 1:
            y_true = tf.argmax(y_true, axis=1)
        
        # Get predicted probabilities
        pred_probs = tf.nn.softmax(y_pred)
        
        # Apply transition matrix correction
        T = tf.constant(self.transition_matrix, dtype=tf.float32)
        corrected_probs = tf.matmul(pred_probs, T)
        
        # Calculate cross-entropy loss with corrected probabilities
        loss = tf.nn.sparse_categorical_crossentropy(y_true, corrected_probs)
        
        return tf.reduce_mean(loss)
    
    def fit(self, X_train, S_train, X_val, S_val, epochs=30, batch_size=128):
        """
        Train the classifier on noisy data and validate.
        
        Args:
            X_train: Training features
            S_train: Noisy training labels
            X_val: Validation features
            S_val: Noisy validation labels
        """
        # Build a new model (to reset weights for multiple runs)
        self.model = self.model_builder()
        
        # Configure the loss function based on whether we use correction
        if self.transition_matrix is not None and self.use_correction:
            loss = self.forward_correction_loss
        else:
            # If no transition matrix provided or correction disabled, use standard CE
            loss = 'sparse_categorical_crossentropy'
        
        # Compile the model
        self.model.compile(
            optimizer='adam',
            loss=loss,
            metrics=['accuracy']
        )
        
        # Train the model
        self.history = self.model.fit(
            X_train, S_train,
            validation_data=(X_val, S_val),
            epochs=epochs,
            batch_size=batch_size,
            verbose=0,  # Silence output for multiple runs
            callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=5,
                    restore_best_weights=True
                )
            ]
        )
        
        return self
    
    def evaluate(self, X_test, Y_test):
        """
        Evaluate the classifier on clean test data.
        
        Args:
            X_test: Test features
            Y_test: Clean test labels
            
        Returns:
            Accuracy on the test set
        """
        # Make predictions
        y_pred = self.model.predict(X_test, verbose=0)
        y_pred_classes = np.argmax(y_pred, axis=1)
        
        # Calculate accuracy
        accuracy = np.mean(y_pred_classes == Y_test)
        
        return accuracy

In [45]:
def run_evaluations(datasets, classifiers, n_runs=3):
    """
    Run evaluations for multiple classifiers on multiple datasets.
    
    Args:
        datasets: Dict of {dataset_name: (Xtr, Str, Xts, Yts)}
        classifiers: List of NoiseRobustClassifier instances
        n_runs: Number of runs with different random splits
        
    Returns:
        Dictionary of results {dataset_name: {classifier_name: (mean_acc, std_acc)}}
    """
    results = {}
    
    for dataset_name, (Xtr, Str, Xts, Yts) in datasets.items():
        print(f"Evaluating on dataset: {dataset_name}")
        dataset_results = {}
        
        for classifier in classifiers:
            print(f"  - Evaluating classifier: {classifier.name}")
            mean_acc, std_acc = evaluate_classifier(classifier, Xtr, Str, Xts, Yts, n_runs)
            dataset_results[classifier.name] = (mean_acc, std_acc)
            print(f"    Accuracy: {mean_acc:.4f} ± {std_acc:.4f}")
        
        results[dataset_name] = dataset_results
    
    return results

In [46]:
def visualize_results(results, title="Classifier Performance Comparison"):
    """
    Create bar plots to visualize classifier performance across datasets.
    
    Args:
        results: Dictionary of results from run_evaluations
        title: Plot title
    """
    # Determine dimensions
    n_datasets = len(results)
    n_classifiers = len(next(iter(results.values())))
    
    # Set up the figure
    plt.figure(figsize=(12, 5 * n_datasets))
    
    # For each dataset
    for i, (dataset_name, dataset_results) in enumerate(results.items()):
        # Create subplot
        plt.subplot(n_datasets, 1, i+1)
        
        # Extract data for plotting
        classifiers = list(dataset_results.keys())
        accuracies = [dataset_results[clf][0] for clf in classifiers]  # mean
        errors = [dataset_results[clf][1] for clf in classifiers]      # std
        
        # Create bar plot
        bars = plt.bar(classifiers, accuracies, yerr=errors, capsize=10)
        
        # Add labels and grid
        plt.title(f"{dataset_name} - Test Accuracy")
        plt.ylabel("Accuracy")
        plt.ylim(0, 1.0)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        
        # Add accuracy values on top of bars
        for bar, acc in zip(bars, accuracies):
            plt.text(bar.get_x() + bar.get_width()/2, 
                     bar.get_height() + 0.01, 
                     f"{acc:.4f}", 
                     ha='center', va='bottom')
    
    plt.tight_layout()
    plt.suptitle(title, fontsize=16, y=1.02)
    plt.show()

def plot_confusion_matrices(classifier, datasets, title="Confusion Matrices"):
    """
    Plot confusion matrices for a classifier on all datasets.
    
    Args:
        classifier: A trained NoiseRobustClassifier instance
        datasets: Dict of {dataset_name: (Xtr, Str, Xts, Yts)}
        title: Plot title
    """
    from sklearn.metrics import confusion_matrix
    
    # Set up the figure
    n_datasets = len(datasets)
    fig, axes = plt.subplots(1, n_datasets, figsize=(5 * n_datasets, 5))
    
    # If only one dataset, make axes iterable
    if n_datasets == 1:
        axes = [axes]
    
    # For each dataset
    for i, (dataset_name, (Xtr, Str, Xts, Yts)) in enumerate(datasets.items()):
        # Get predictions
        y_pred = classifier.model.predict(Xts, verbose=0)
        y_pred_classes = np.argmax(y_pred, axis=1)
        
        # Compute confusion matrix
        cm = confusion_matrix(Yts, y_pred_classes)
        
        # Plot confusion matrix
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i])
        axes[i].set_title(f"{dataset_name} - {classifier.name}")
        axes[i].set_xlabel("Predicted Label")
        axes[i].set_ylabel("True Label")
    
    plt.tight_layout()
    plt.suptitle(title, fontsize=16, y=1.02)
    plt.show()

In [57]:
def main():
    """
    Main execution function demonstrating the evaluation framework.
    """
    # Load datasets
    print("Loading datasets...")
    
    # FashionMNIST 0.3 noise
    dataset1 = np.load('/Users/Aidar Alimbayev/Documents/projects_aidar_mac/ml805_labs/Assingment 2 /datasets/FashionMNIST0.3.npz')
    # Load dataset
    X_train = dataset1['X_tr']  # Training features
    S_train = dataset1['S_tr']  # Noisy labels for training
    X_test = dataset1['X_ts']  # Test features
    Y_test = dataset1['Y_ts']  # Clean labels for testing
    
    Xtr1, Str1 = dataset1['X_tr'], dataset1['S_tr']
    Xts1, Yts1 = dataset1['X_ts'], dataset1['Y_ts']
    
    # Preprocess data (normalize pixel values)
    Xtr1 = Xtr1.astype('float32') / 255.0
    Xts1 = Xts1.astype('float32') / 255.0
    
    
    
    
    # FashionMNIST 0.6 noise
    dataset2 = np.load('/Users/Aidar Alimbayev/Documents/projects_aidar_mac/ml805_labs/Assingment 2 /datasets/FashionMNIST0.6.npz')
    Xtr2, Str2 = dataset2['X_tr'], dataset2['S_tr']
    Xts2, Yts2 = dataset2['X_ts'], dataset2['Y_ts']
    
    # Preprocess data
    Xtr2 = Xtr2.astype('float32') / 255.0
    Xts2 = Xts2.astype('float32') / 255.0
    
    # CIFAR - unknown transition matrix
    dataset3 = np.load('/Users/Aidar Alimbayev/Documents/projects_aidar_mac/ml805_labs/Assingment 2 /datasets/CIFAR10.npz')
    Xtr3, Str3 = dataset3['X_tr'], dataset3['S_tr']
    Xts3, Yts3 = dataset3['X_ts'], dataset3['Y_ts']
    
    # Preprocess data
    Xtr3 = Xtr3.astype('float32') / 255.0
    Xts3 = Xts3.astype('float32') / 255.0
    
    # Define transition matrices
    T_fashion_03 = np.array([
        [0.7, 0.3, 0, 0],
        [0, 0.7, 0.3, 0],
        [0, 0, 0.7, 0.3],
        [0.3, 0, 0, 0.7]
    ])
    
    T_fashion_06 = np.array([
        [0.4, 0.2, 0.2, 0.2],
        [0.2, 0.4, 0.2, 0.2],
        [0.2, 0.2, 0.4, 0.2],
        [0.2, 0.2, 0.2, 0.4]
    ])
    
    # Create transition matrix estimator for CIFAR
    # This is a placeholder - you'll implement your own estimator
    T_cifar_estimated = estimate_transition_matrix(Xtr3, Str3)
    
    # Organize datasets
    datasets = {
        'FashionMNIST_0.3': (Xtr1, Str1, Xts1, Yts1),
        'FashionMNIST_0.6': (Xtr2, Str2, Xts2, Yts2),
        'CIFAR': (Xtr3, Str3, Xts3, Yts3)
    }
    
    # Define model builders
    def build_fashion_mnist_cnn():
        model = models.Sequential([
            layers.Reshape((28, 28, 1), input_shape=(28, 28)),
            layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(4)
        ])
        return model
    
    def build_cifar_cnn():
        model = models.Sequential([
            layers.Input(shape=(32, 32, 3)),
            layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
            layers.BatchNormalization(),
            layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.2),
            layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.3),
            layers.Flatten(),
            layers.Dense(256, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(4)
        ])
        return model
    
    # Create classifiers
    classifiers = [
        # Standard CNN without correction
        NoiseRobustClassifier(
            name="Standard CNN",
            model_builder=build_fashion_mnist_cnn,
            use_correction=False
        ),
        
        # Forward Correction with known transition matrix
        NoiseRobustClassifier(
            name="Forward Correction",
            model_builder=build_fashion_mnist_cnn,
            transition_matrix=T_fashion_03,
            use_correction=True
        ),
        
        # Mean Absolute Error loss (inherently robust to noise)
        NoiseRobustClassifier(
            name="MAE Loss",
            model_builder=build_fashion_mnist_cnn,
            use_correction=False
        )
    ]
    
    # Run evaluations
    results = run_evaluations(datasets, classifiers, n_runs=3)
    
    # Visualize results
    visualize_results(results)
    
    # Plot confusion matrices for the best classifier
    best_classifier = classifiers[1]  # Assuming Forward Correction is best
    plot_confusion_matrices(best_classifier, datasets)

# Placeholder for transition matrix estimator (you will implement this)
def estimate_transition_matrix(X, S, num_classes=4):
    """
    Estimate the transition matrix from noisy data.
    
    Args:
        X: Features
        S: Noisy labels
        num_classes: Number of classes
        
    Returns:
        Estimated transition matrix
    """
    # This is a placeholder - you will implement your own estimation method
    # For now, just return an identity matrix (no correction)
    return np.eye(num_classes)

if __name__ == "__main__":
    main()

Loading datasets...
Evaluating on dataset: FashionMNIST_0.3
  - Evaluating classifier: Standard CNN


/opt/anaconda3/envs/assing_2/lib/python3.12/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


KeyboardInterrupt: 

In [ ]:
def estimate_transition_matrix_anchor_points(X, S, num_classes=4, anchor_threshold=0.9):
    """
    Estimate transition matrix using anchor points method.
    
    Args:
        X: Features
        S: Noisy labels
        num_classes: Number of classes
        anchor_threshold: Confidence threshold for anchor points
        
    Returns:
        Estimated transition matrix
    """
    from sklearn.model_selection import train_test_split
    
    # Split data for initial model training
    X_train, X_val, S_train, S_val = train_test_split(X, S, test_size=0.2)
    
    # Create and train a base model
    base_model = models.Sequential([
        layers.Input(shape=X.shape[1:]),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    base_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
    base_model.fit(X_train, S_train, validation_data=(X_val, S_val), 
                   epochs=10, verbose=0)
    
    # Get predicted probabilities
    pred_probs = base_model.predict(X)
    
    # Initialize transition matrix
    T = np.zeros((num_classes, num_classes))
    
    # For each true class, find anchor points and estimate transition probabilities
    for i in range(num_classes):
        # Find anchor points: points where the model is very confident about class i
        anchor_idx = np.where(pred_probs[:, i] >= anchor_threshold)[0]
        
        if len(anchor_idx) > 0:
            # Count occurrences of each noisy label among anchor points
            for j in range(num_classes):
                class_j_count = np.sum(S[anchor_idx] == j)
                T[i, j] = class_j_count / len(anchor_idx)
        else:
            # If no anchor points found, use a fallback (identity or prior knowledge)
            T[i, i] = 0.7  # Assuming diagonal dominance as a prior
            T[i, (i+1) % num_classes] = 0.3  # Some off-diagonal probability
    
    # Ensure each row sums to 1
    row_sums = T.sum(axis=1, keepdims=True)
    T = T / row_sums
    
    return T

# Draft test

In [13]:
%pip install matplotlib


  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 10.2 MB/s eta 0:00:00 0:00:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 22.6 MB/s eta 0:00:00
Using cached pyparsing-3.2.1-py3-none-any.whl (107 kB)
Note: you may need to restart the kernel to use updated packages.


In [14]:
import numpy as np
import matplotlib.pyplot as plt

# Define the transition matrix A
A = np.array([[0.2987, 0],
              [0, 0.134]])

B = np.array([[0.5, 0.71],
              [0.352, 0.5],
              [0.352, 0.5],
              [0.5, 0.71]])

In [16]:
A

array([[0.2987, 0.    ],
       [0.    , 0.134 ]])

In [17]:
B

array([[0.5  , 0.71 ],
       [0.352, 0.5  ],
       [0.352, 0.5  ],
       [0.5  , 0.71 ]])

In [26]:
M = np.array([[0.5, 0.352],
              [0.71, 0.5]])

In [27]:
C =  M @ A @ M.T

print(C)    

[[0.09127814 0.1296225 ]
 [0.1296225  0.18407467]]


In [28]:
C = np.matmul(A, M)
C

array([[0.14935  , 0.1051424],
       [0.09514  , 0.067    ]])

In [29]:
N = np.array([[0.352, 0.5],
              [0.5, 0.71]])

In [33]:
C = N @ A @ N.T
C

array([[0.07051012, 0.1001412 ],
       [0.1001412 , 0.1422244 ]])